# Algoritmo Apriori - Reglas de Asociación

Desarrollo de un análisis de asociación en Python usando el algoritmo a priori para derivar reglas de la forma {A, B} -> {C}, usando los datos del problema Instacart Market [link](https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis/notebook) en Kaggle, teniendo en cuenta las siguientes caracteristicas:

* minimum Support: 1%
*     confidence: 70%

## 1. Librerias python 

Se procede al llamado de las librerias python requeridas para desarrollar la tarea

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby, permutations
from collections import Counter
from IPython.display import display

## 2. Carga de datos en Memoria

los datos se deben descargar del siguientes [link](https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis/data) y almacenados en una carpeta llamada raw_data junto a este notebook, adicionalmente se observan algunos datos de la información cargada

In [2]:
orders   = pd.read_csv("raw_data/order_products__prior.csv")

display(orders.head())
print('Número de Registros: {:,}'.format(orders.shape[0]))
print('Número de Columnas : {}'.format(orders.shape[1]))

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


Número de Registros: 32,434,489
Número de Columnas : 4


## 3. Eliminación de columnas 

Se procede a eliminar columnas de los datos cargados con el fin de liberar memoria

In [3]:
orders = orders.drop(['add_to_cart_order', 'reordered'], axis = 1)
display(orders.head())
print('Número de Registros: {:,}'.format(orders.shape[0]))
print('Número de Columnas : {}'.format(orders.shape[1]))

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


Número de Registros: 32,434,489
Número de Columnas : 2


## 4. Eliminacón ordenes de compra

Para el problema que estamos tratando de resolver (es decir, encontrar relaciones del tipo {A, B} -> {C} ), solo se puede desarrollar con ordenes de al menos 3 elementos, por lo cual se procede a eliminar ordenes con menor numero de productos

In [4]:
value_counts = orders['order_id'].value_counts()
to_remove = value_counts[value_counts < 3].index

orders = orders[~orders.order_id.isin(to_remove)]

display(orders.head())
print('Número de Registros: {:,}'.format(orders.shape[0]))
print('Número de Columnas : {}'.format(orders.shape[1]))


del value_counts
del to_remove

,order_id,product_id
0,2,33120
1,2,28985
2,2,9327
3,2,45918
4,2,30035


Número de Registros: 31,903,755
Número de Columnas : 2


## 5. Cálculo de c1

In [5]:
serie_c1 = orders['product_id'].value_counts()

frame = { 'counta': serie_c1}

table_c1 = pd.DataFrame(frame)

table_c1['item_set'] = table_c1.index
table_c1['sup_count'] = table_c1.counta

table_c1.reset_index(inplace=True)

table_c1 = table_c1.drop(['index', 'counta'], axis = 1)

display(table_c1.head())

print('Número de Registros: {:,}'.format(table_c1.shape[0]))
print('Número de Columnas : {}'.format(table_c1.shape[1]))

,item_set,sup_count
0,24852,465141
1,13176,369297
2,21137,260411
3,21903,237743
4,47209,211206


Número de Registros: 49,672
Número de Columnas : 2


## 6. Cálculo de L1

El ejercicio establece un minimum Support: 1%, pero con esta condición el filtro encuentra 2 registros, por lo tanto se prueba con:

minimum Support: 0.5%

minimum Support: 0.04%

Nota: Para el cálculo de minimum Support Count se usa la formula explicada en la pagina 5 del [pdf](https://www3.cs.stonybrook.edu/~cse634/lecture_notes/07apriori.pdf)

In [6]:
import math

min_sup_count1 = math.ceil(orders.shape[0] * 0.01)
min_sup_count2 = math.ceil(orders.shape[0] * 0.005)
min_sup_count3 = math.ceil(orders.shape[0] * 0.0004)

print('      Número Total de ordenes: {:,}'.format(orders.shape[0]))
print('minimum Support Count    (1%): {:,}'.format(min_sup_count1))
print('minimum Support Count  (0.5%): {:,}'.format(min_sup_count2))
print('minimum Support Count (0.04%): {:,}\n'.format(min_sup_count3))

table_L1 = table_c1.loc[table_c1['sup_count'] >= min_sup_count3]

print(table_L1)

print('\nNúmero de Registros: {:,}'.format(table_L1.shape[0]))
print('Número de Columnas : {}'.format(table_L1.shape[1]))

      Número Total de ordenes: 31,903,755
minimum Support Count    (1%): 319,038
minimum Support Count  (0.5%): 159,519
minimum Support Count (0.04%): 12,762

     item_set  sup_count
0       24852     465141
1       13176     369297
2       21137     260411
3       21903     237743
4       47209     211206
5       47766     174963
6       47626     151180
7       26209     139458
8       16797     139240
9       27966     135747
10      27845     134573
11      22935     112497
12      24964     108941
13      45007     104082
14      39275      97701
15      49683      96532
16      28204      88608
17       5876      86982
18       8277      84217
19      40706      83835
20       4920      81454
21      30391      79925
22      45066      78949
23      42265      75876
24      44632      74523
25      19057      74424
26      49235      73589
27       4605      72832
28      37646      72248
29      17794      72224
..        ...        ...
304      3599      13617
305     17706   

## 7. Cálculo de c2

Para desarrollar esta tarea es necesario crear una función auxiliar que calcula las combinaciones de L1

In [7]:
# Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item, lon):
    for item_pair in combinations(order_item, lon):
        yield item_pair

gen_obj = get_item_pairs(table_L1['item_set'].values.tolist(), 2)

item_set1 = []
item_set2 = []
for el in gen_obj:
    #print(el)
    item_set1.append(el[0])
    item_set2.append(el[1])

table_c2 = pd.DataFrame(columns=['item_set1', 'item_set2', 'sup_count'])

for i, u in enumerate(item_set1):
    table_c2.loc[i, 'item_set1'] = "{}".format(u)
    table_c2.loc[i, 'item_set2'] = item_set2[i]
    
    tbl_one = orders[(orders.product_id == u)]
    tbl_two = tbl_one.merge(orders[(orders.product_id == item_set2[i])], on='order_id')
    
    table_c2.loc[i, 'sup_count'] = tbl_two.shape[0]

print(table_c2)

print('\nNúmero de Registros: {:,}'.format(table_c2.shape[0]))
print('Número de Columnas : {}'.format(table_c2.shape[1]))

      item_set1 item_set2 sup_count
0         24852     13176      1172
1         24852     21137     56058
2         24852     21903     51339
3         24852     47209     31189
4         24852     47766     53322
5         24852     47626     40841
6         24852     26209     31753
7         24852     16797     41187
8         24852     27966     22876
9         24852     27845     31574
10        24852     22935     16690
11        24852     24964     19092
12        24852     45007     19303
13        24852     39275     18958
14        24852     49683     32069
15        24852     28204     33856
16        24852      5876     10861
17        24852      8277     17394
18        24852     40706     18713
19        24852      4920     24561
20        24852     30391     12533
21        24852     45066     28319
22        24852     42265     16744
23        24852     44632     15769
24        24852     19057     13035
25        24852     49235     13124
26        24852      4605   

## 8. Cálculo de L2

In [8]:
print('      Número Total de ordenes: {:,}'.format(orders.shape[0]))
print('minimum Support Count (0.04%): {:,}\n'.format(min_sup_count3))

table_L2 = table_c2.loc[table_c2['sup_count'] >= min_sup_count3]

print(table_L2)

print('\nNúmero de Registros: {:,}'.format(table_L2.shape[0]))
print('Número de Columnas : {}'.format(table_L2.shape[1]))

      Número Total de ordenes: 31,903,755
minimum Support Count (0.04%): 12,762

     item_set1 item_set2 sup_count
1        24852     21137     56058
2        24852     21903     51339
3        24852     47209     31189
4        24852     47766     53322
5        24852     47626     40841
6        24852     26209     31753
7        24852     16797     41187
8        24852     27966     22876
9        24852     27845     31574
10       24852     22935     16690
11       24852     24964     19092
12       24852     45007     19303
13       24852     39275     18958
14       24852     49683     32069
15       24852     28204     33856
17       24852      8277     17394
18       24852     40706     18713
19       24852      4920     24561
21       24852     45066     28319
22       24852     42265     16744
23       24852     44632     15769
24       24852     19057     13035
25       24852     49235     13124
26       24852      4605     21157
27       24852     37646     16932
28       

## 9. Cálculo de c3

In [9]:
list_c3a = table_L2['item_set1'].values.tolist()
list_c3b = table_L2['item_set2'].values.tolist()

table_c3 = pd.DataFrame(columns=['item_set1', 'item_set2', 'item_set3', 'sup_count'])

for j, d in enumerate(list_c3a):
    ultima = len(list_c3a) - 1 - list_c3a[::-1].index(d)
    for s in range(j, ultima, 1):
        g = table_c3.shape[0]
        table_c3.loc[g, 'item_set1'] = d
        table_c3.loc[g, 'item_set2'] = list_c3b[j]
        table_c3.loc[g, 'item_set3'] = list_c3b[s+1]
        
        tab_one  = orders[(orders.product_id == int(d))]
        tab_two  = tab_one.merge(orders[(orders.product_id == int(list_c3b[j]))], on='order_id')
        tab_thre = tab_two.merge(orders[(orders.product_id == int(list_c3b[s+1]))], on='order_id')
        
        table_c3.loc[g, 'sup_count'] = tab_thre.shape[0]

print(table_c3)        

print('\nNúmero de Registros: {:,}'.format(table_c3.shape[0]))
print('Número de Columnas : {}'.format(table_c3.shape[1]))

     item_set1 item_set2 item_set3 sup_count
0        24852     21137     21903      9452
1        24852     21137     47209      6881
2        24852     21137     47766      8700
3        24852     21137     47626      5490
4        24852     21137     26209      5405
5        24852     21137     16797       318
6        24852     21137     27966      6410
7        24852     21137     27845      6688
8        24852     21137     22935      3074
9        24852     21137     24964      3280
10       24852     21137     45007      3206
11       24852     21137     39275      4682
12       24852     21137     49683      4955
13       24852     21137     28204      3761
14       24852     21137      8277      3383
15       24852     21137     40706      3911
16       24852     21137      4920      4432
17       24852     21137     45066      4243
18       24852     21137     42265      3361
19       24852     21137     44632      2278
20       24852     21137     19057      2904
21       2

## 10. Cálculo de L3

In [12]:
print('      Número Total de ordenes: {:,}'.format(orders.shape[0]))
print('minimum Support Count (0.04%): {:,}\n'.format(min_sup_count3))

table_L3 = table_c3.loc[table_c3['sup_count'] >= min_sup_count3]

print(table_L3)

print('\nNúmero de Registros: {:,}'.format(table_L3.shape[0]))
print('Número de Columnas : {}'.format(table_L3.shape[1]))

      Número Total de ordenes: 31,903,755
minimum Support Count (0.04%): 12,762

    item_set1 item_set2 item_set3 sup_count
904     13176     21137     47209     15066

Número de Registros: 1
Número de Columnas : 4


## 11. Cálculo Reglas de Asociación

para el cálculo de las reglas de sasociacón solo se tendra en cuenta las relaciones del tipo {A, B} -> {C}

In [79]:
list_ar1 = table_L3['item_set1'].values.tolist()
list_ar2 = table_L3['item_set2'].values.tolist()
list_ar3 = table_L3['item_set3'].values.tolist()
list_spc = table_L3['sup_count'].values.tolist()

table_ar = pd.DataFrame(columns=['item_set1', 'item_set2', '->', 'item_set3', 'freq_abc', 'freq_ab', 'Confidence'])

for qu, z in enumerate(list_ar1):
    f = 3 * qu
    table_ar.loc[f, 'item_set1']   = z
    table_ar.loc[f, 'item_set2']   = list_ar2[qu]
    table_ar.loc[f, '->']          = '->'
    table_ar.loc[f, 'item_set3']   = list_ar3[qu]
    table_ar.loc[f, 'freq_abc']    = list_spc[qu]
    table_ar.loc[f, 'freq_ab']     = table_L2[(table_L2['item_set1'] == str(z)) & (table_L2['item_set2'] == list_ar2[qu])].iloc[0]['sup_count']
    
    table_ar.loc[f+1, 'item_set1'] = z
    table_ar.loc[f+1, 'item_set2'] = list_ar3[qu]
    table_ar.loc[f+1, '->']        = '->'
    table_ar.loc[f+1, 'item_set3'] = list_ar2[qu]
    table_ar.loc[f+1, 'freq_abc']  = list_spc[qu]
    table_ar.loc[f+1, 'freq_ab']   = table_L2[(table_L2['item_set1'] == str(z)) & (table_L2['item_set2'] == list_ar3[qu])].iloc[0]['sup_count']
    
    table_ar.loc[f+2, 'item_set1'] = list_ar2[qu]
    table_ar.loc[f+2, 'item_set2'] = list_ar3[qu]
    table_ar.loc[f+2, '->']        = '->'
    table_ar.loc[f+2, 'item_set3'] = z
    table_ar.loc[f+2, 'freq_abc']  = list_spc[qu]
    table_ar.loc[f+2, 'freq_ab']   = table_L2[(table_L2['item_set1'] == str(list_ar2[qu])) & (table_L2['item_set2'] == list_ar3[qu])].iloc[0]['sup_count']

    
table_ar['Confidence'] = (table_ar['freq_abc'] / table_ar['freq_ab'])

table_ar['Confidence'] = pd.Series(["{0:.2f}%".format(val * 100) for val in table_ar['Confidence']], index = table_ar.index)

print(table_ar)


  item_set1 item_set2  -> item_set3 freq_abc freq_ab Confidence
0     13176     21137  ->     47209    15066   61482     24.50%
1     13176     47209  ->     21137    15066   62255     24.20%
2     21137     47209  ->     13176    15066   40761     36.96%


## 12. Asociando Datos

In [80]:
products = pd.read_csv("raw_data/products.csv")

display(products.head(10))
print('Número de Registros: {:,}'.format(products.shape[0]))
print('Número de Columnas : {}\n'.format(products.shape[1]))


table_ar['item_set1'] = table_ar['item_set1'].astype(int)
table_ar['item_set2'] = table_ar['item_set2'].astype(int)
table_ar['item_set3'] = table_ar['item_set3'].astype(int)

rename_dict = products.set_index('product_id').to_dict()['product_name']

table_ar['item_set1'] = table_ar['item_set1'].replace(rename_dict)
table_ar['item_set2'] = table_ar['item_set2'].replace(rename_dict)
table_ar['item_set3'] = table_ar['item_set3'].replace(rename_dict)

display(table_ar.head())

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
5,6,Dry Nose Oil,11,11
6,7,Pure Coconut Water With Orange,98,7
7,8,Cut Russet Potatoes Steam N' Mash,116,1
8,9,Light Strawberry Blueberry Yogurt,120,16
9,10,Sparkling Orange Juice & Prickly Pear Beverage,115,7


Número de Registros: 49,688
Número de Columnas : 4



,item_set1,item_set2,->,item_set3,freq_abc,freq_ab,Confidence
0,Bag of Organic Bananas,Organic Strawberries,->,Organic Hass Avocado,15066,61482,24.50%
1,Bag of Organic Bananas,Organic Hass Avocado,->,Organic Strawberries,15066,62255,24.20%
2,Organic Strawberries,Organic Hass Avocado,->,Bag of Organic Bananas,15066,40761,36.96%


## 13. Conclusiones

* El minimum Support propuesto en el problema inicial de 1%, no es posible alcanzarlos para las relaciones {A, B} -> {C} ya que el conteo es muy alto debido a la cantidad de registros.


* El proceso de cálculo de C2 es el mas complejo debido al alto volumen de datos, por lo cual es necesario revisar otras técnicas de procesado de información. 

## Referencias (webgrafia)
* https://www.kaggle.com/datatheque/association-rules-mining-market-basket-analysis
* http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
* https://www3.cs.stonybrook.edu/~cse634/lecture_notes/07apriori.pdf
* https://data36.com/pandas-tutorial-2-aggregation-and-grouping/
* https://pythonspot.com/pandas-filter/
* https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
* https://www.ritchieng.com/pandas-multi-criteria-filtering/
* https://www.interviewqs.com/ddi_code_snippets/add_new_col_df_default_value
* https://www.analyticslane.com/2019/05/10/operaciones-de-filtrado-de-dataframe-con-pandas-en-base-a-los-valores-de-las-columnas/
* http://www2.cs.uregina.ca/~dbd/cs831/notes/itemsets/itemset_apriori.html